# Airfoil Flap Calculations Notebook

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
from time import sleep

from ICARUS.database import DB
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
airfoil = DB.get_airfoil('NACA0009')

fig = plt.figure()
ax = fig.add_subplot(111)

# airfoil.plot(ax=ax)
airfoil_flap = airfoil.flap(
    flap_hinge_chord_percentage= 0.63,
    flap_angle= 10.,
    chord_extension= 1., 
)

# airfoil_flap.plot(ax=ax, overide_color = 'green', scatter = True)
airfoil_flap.repanel_from_internal(120, "cosine")
airfoil_flap.plot(ax=ax, overide_color = 'red', scatter=True)
airfoil_flap.repanel_spl(120, 0)
airfoil_flap.plot(ax=ax, overide_color = 'blue', scatter=True)

In [ ]:
from ICARUS.core.units import calc_reynolds
import numpy as np
from ICARUS.core.types import FloatArray

# PARAMETERS FOR ESTIMATION
chord_max: float = 0.5
chord_min: float = 0.1
u_max: float = 35.0
u_min: float = 5.0
viscosity: float = 1.56e-5
speed_of_sound: float = 340.3

# MACH ESTIMATION
mach_max: float = 0.0
# mach_min: float = calc_mach(10, speed_of_sound)
# mach: FloatArray = np.linspace(mach_max, mach_min, 10)
MACH: float = mach_max

# REYNOLDS ESTIMATION
RE_MIN = 8e4
RE_MAX = 1.5e6
NUM_BINS = 12
REYNOLDS_BINS = np.logspace(-2.2, 0, NUM_BINS) * (RE_MAX - RE_MIN) + RE_MIN
reynolds = REYNOLDS_BINS

# ANGLE OF ATTACK SETUP
aoa_min: float = -10
aoa_max: float = 16
num_of_angles: int = int((aoa_max - aoa_min) * 2 + 1)
angles: FloatArray = np.linspace(
    start=aoa_min,
    stop=aoa_max,
    num=num_of_angles,
)

# Transition to turbulent Boundary Layer
ftrip_up: dict[str, float] = {"pos": 0.2, "neg": 0.1}
ftrip_low: dict[str, float] = {"pos": 0.1, "neg": 0.2}
Ncrit = 9

In [ ]:
from ICARUS.computation.solvers.solver import Solver
# # Xfoil
from ICARUS.computation.solvers.Xfoil.xfoil import Xfoil

print(f"\nRunning airfoil {airfoil}\n")

for flap_angle in np.arange(7.5, 35, 2.5):
    fig = plt.figure()
    ax = fig.add_subplot(111)

    airfoil_flap = airfoil.flap(
        flap_hinge_chord_percentage= 0.63,
        flap_angle=flap_angle,
        chord_extension= 1., 
    )
    airfoil_flap.repanel_from_internal(120, "cosine")
    airfoil_flap.repanel_spl(120, 1e-6)

    airfoil_flap.plot(scatter=True, ax = ax)
    airfoil_flap.plot( ax = ax)
    plt.show(block=True)
    xfoil: Solver = Xfoil()

    # Import Analysis
    analysis: str = xfoil.get_analyses_names()[1]  # Run
    xfoil.select_analysis(analysis)

    # Get Options
    options = xfoil.get_analysis_options(verbose=False)
    solver_parameters = xfoil.get_solver_parameters()

    # Set Options
    options.airfoil = airfoil_flap
    options.mach = MACH
    options.reynolds = reynolds
    options.min_aoa = aoa_min
    options.max_aoa = aoa_max
    options.aoa_step = 0.5

    # Set Solver Options
    solver_parameters.max_iter = 200
    solver_parameters.Ncrit = 9
    solver_parameters.xtr = (.05, .05)
    solver_parameters.print = False
    solver_parameters.repanel_n = 120

    xfoil.define_analysis(options, solver_parameters)
    xfoil.print_analysis_options()

    # RUN
    xfoil.execute(parallel=False)
    # Get polar
    polar = DB.foils_db.get_polars(airfoil_flap.name.upper())
    fig = polar.plot()
    fig.show()
    plt.show(block=True)


In [ ]:
airfs = []
for flap_angle in np.arange(0, 35, 2.5):
    try:
        airfoil_flap = airfoil.flap(
            flap_hinge_chord_percentage= 0.63,
            flap_angle=flap_angle,
            chord_extension= 1., 
        )
        airfs.append(airfoil_flap)
        polar = DB.foils_db.get_polars(airfoil_flap.name.upper())
        fig = polar.plot()
        fig.show()
    except:
        pass

airf_names = [airf.name for airf in airfs]

In [ ]:
airf_names

In [ ]:
from ICARUS.visualization.airfoil.db_polars import plot_airfoils_polars

plot_airfoils_polars(
    airf_names[1:],
    reynolds= 4e5,
    solvers=["Xfoil"]
)